In [1]:
%matplotlib inline

# logistic regression model
Starter code for logistic regression model to solve OCR task 
with MNIST in TensorFlow
MNIST dataset: yann.lecun.com/exdb/mnist/


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

In [3]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 25

In [4]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('../data/mnist', one_hot=True) 

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [5]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
X = tf.placeholder(dtype=tf.float32, shape=[batch_size, 784], name='X')
Y = tf.placeholder(dtype=tf.float32, shape=[batch_size, 10], name='Y')

In [6]:
# Step 3: create weights and bias
# weights and biases are initialized to 0
# shape of w depends on the dimension of X and Y so that Y = X * w + b
# shape of b depends on Y
w = tf.Variable(tf.random_normal(shape=[784, 10], mean=0, stddev=1), name='w')
b = tf.Variable(tf.random_normal(shape=[1, 10], mean=0, stddev=1), name='b')

In [7]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE
logits = tf.matmul(X, w) + b

In [10]:
# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch
sce = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss = tf.reduce_mean(sce)

In [11]:
# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [19]:
# Step 7: run
ginit = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(ginit)
    n_batches = int(mnist.train.num_examples / batch_size)
    for _ in range(n_epochs):
        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size=batch_size)
            _, batch_loss = sess.run([optimizer, loss], {X: X_batch, Y: Y_batch})
            # print batch_loss
    # step 8: test model
    nt_batches = int(mnist.test.num_examples / batch_size)
    n_total_correct = 0
    for _ in range(nt_batches):
        X_test, Y_test = mnist.test.next_batch(batch_size=batch_size)
        _, batch_loss, batch_logits = sess.run([optimizer, loss, logits], {X: X_test, Y: Y_test})
        preds = tf.nn.softmax(batch_logits)
        correct_preds = tf.equal(tf.arg_max(preds, 1), tf.arg_max(Y_test, 1))
        n_correct_pred = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        n_total_correct += sess.run(n_correct_pred)
    print 'Accuracy: %s' % (n_total_correct / mnist.test.num_examples)

Accuracy: 0.8176
